# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></div><div class="lev2 toc-item"><a href="#Parse-input" data-toc-modified-id="Parse-input-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Parse input</a></div><div class="lev1 toc-item"><a href="#Overview-visualization" data-toc-modified-id="Overview-visualization-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Overview visualization</a></div><div class="lev1 toc-item"><a href="#Differential-expression" data-toc-modified-id="Differential-expression-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Differential expression</a></div><div class="lev2 toc-item"><a href="#Without-batch-correction" data-toc-modified-id="Without-batch-correction-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Without batch correction</a></div><div class="lev2 toc-item"><a href="#With-batch-correction" data-toc-modified-id="With-batch-correction-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>With batch correction</a></div><div class="lev1 toc-item"><a href="#Run-ComBat-to-adjust" data-toc-modified-id="Run-ComBat-to-adjust-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Run ComBat to adjust</a></div><div class="lev2 toc-item"><a href="#ComBat-modelling-condition" data-toc-modified-id="ComBat-modelling-condition-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>ComBat modelling condition</a></div><div class="lev3 toc-item"><a href="#Statistics" data-toc-modified-id="Statistics-411"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Statistics</a></div><div class="lev2 toc-item"><a href="#ComBat-not-modelling-biological-factor" data-toc-modified-id="ComBat-not-modelling-biological-factor-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>ComBat not modelling biological factor</a></div><div class="lev3 toc-item"><a href="#Statistics" data-toc-modified-id="Statistics-421"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Statistics</a></div><div class="lev1 toc-item"><a href="#Estimate-surrogate-variation" data-toc-modified-id="Estimate-surrogate-variation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Estimate surrogate variation</a></div><div class="lev1 toc-item"><a href="#Summarize-significance" data-toc-modified-id="Summarize-significance-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Summarize significance</a></div>

# Analyzing data

In this part we explore how well we can distinguish spike-in protein despite batch effects when handling the batch effects in a variety of ways. The following are tried:


## Dependencies

This analysis is purely performed in R. It uses the following R- and BioConductor-packages:

* ggplot2
* ggfortify
* ggdendro
* gridExtra
* sva

It also uses custom scripts for generation and visualization of the PCA components.

## Batch effect approaches

###  No batch effect compensation

Contrasts are performed as there were no batch effect

### Batch effect compensation build into the statistical model

The data is unchanged but the batch effect is added as a confounding factor in the ANOVA model.

### Removing batch effects using ComBat

ComBat is a tool attempting to estimate and remove batch effects directly from the data. It produces a new dataset for which contrasts are performed without using confounding factor.

Batch effects are modelled in two ways. In one case, the ComBat model receives information about the different conditions present in the data. In the other, the batch effects are modelled without it. 

One voiced concern about ComBat is that it can give 'wishful results' creating differences where there are none based on the modelled conditions. This is the reasoning behind running it without the biological condition as included input.

### Estimating surrogate variables and using these to remove batch effects

When running ComBat we use a known factor to model batch effects. When estimating the surrogate variables we instead estimate it directly from the data. P-values are then estimated using this information in addition to the raw data.

## Visualizations

Here, I have used the PCA plot as my main tool to visualize the biological effect and the batch effect. In the unadjusted data, the batch effect clearly appears as the strongest effect being modelled by the principal component capturing 45% of the variation. The actual spike-in difference is modelled by the second, capturing 23% of the variance. 

After processing the data using ComBat the spike-in condition appears with the strongest effect showing 37% and 36% for data modelled with and without the condition levels.


## Conclusions

For this dataset, batch correction methods all resulted in a higher number of true positives  being detected. The problem was that the number of false positives increased 2-4 times compared to without applying the batch correction. In this particular dataset, it is hard to argue that one should use batch correction.

Out of the different methods used simply including the batch effect as a confounding factor in the statistical model was the method which included the least number of extra false positives. Although, the same effect could likely have been achieved by slightly adjusting the original threshold value.

It should be noted though that this is a quite 'sterile' simulated dataset. If using a more complex dataset, perhaps one where the difference between biological samples is less clear, this it might be tha

# Setup

In [1]:
run <- "example_data"
expression_fp <- paste0(run, "/full_quant.tsv")
design_fp <- paste0(run, "/design.tsv")

In [3]:
source("util_scripts/proteomics_multivariate_vis.R")
source("util_scripts/visualization_utils.R")
#source("util_scripts/proteomics_stats.R")


In [2]:
library(ggplot2)
library(ggfortify)
library(ggdendro)
library(gridExtra)
library(sva)

source("util_scripts/proteomics_multivariate_vis.R")
source("util_scripts/visualization_utils.R")
#source("util_scripts/proteomics_stats.R")

Warning message:
“replacing previous import by ‘tibble::as_tibble’ when loading ‘ggfortify’”

ERROR: Error in library(sva): there is no package called ‘sva’


In [ ]:
plot_pca <- function(data_m, design_m, pc1, pc2, color_factor, colors, custom_names, legend=T, title_app="", cont_scale=F) {
    
    if (!cont_scale) {
        getPalette <- colorRampPalette(RColorBrewer::brewer.pal(9, "Set1"))        
    }
    title <- paste0("PCA, PC", pc1, " vs PC", pc2, " ", title_app)
    plt <- make_expression_pca(data_m, design_m, color_factor=color_factor, title=title, 
                               pca_axis1=pc1, pca_axis2=pc2, show_labels=T,
                               only_text=T, color_text=T, custom_names=custom_names)
    if (!cont_scale) {
        plt <- plt + scale_color_manual(values=getPalette(colors))        
    }
    
    if (!legend) {
        plt <- plt + theme(legend.position="none")
    }
    
    plt
}

## Parse input

In [ ]:
design_df <- read.csv(design_fp, sep="\t")
design_df$sample <- design_df$name
design_df$batch <- as.factor(design_df$batch)
print(design_df)

In [ ]:
raw_data_df <- read.csv(expression_fp, sep="\t")
head(raw_data_df)
raw_data_df$peptide <- as.character(raw_data_df$peptide)
raw_data_df$protein <- as.character(raw_data_df$protein)
data_df <- log2(raw_data_df[, as.character(design_df$name)])
head(data_df)

In [ ]:
parse_annot <- function(annot_string) {
    
    fields <- unlist(strsplit(as.character(annot_string), "/"))
    annot <- sapply(fields, function(field) { unlist(strsplit(field, "_"))[[2]] })
    
    if (annot == "SOLTU") {
        TRUE
    }
    else if (annot == "ECOLI") {
        FALSE
    }
    else {
        stop(paste0("Unknown annotation: ", annot))
    }
}

In [ ]:
spike_col <- sapply(as.character(raw_data_df[, "protein"]), parse_annot)
head(spike_col)


# Overview visualization

Clustering of unadjusted based on main principal components. Conditions are distinguished 'a' and 'b', while batches are distinguished by color. 

In [ ]:
p1_1 <- plot_pca(data_df, design_df, 1, 2, "batch", colors=4, custom_names=design_df$name, legend=F)
p1_2 <- plot_pca(data_df, design_df, 3, 4, "batch", colors=4, custom_names=design_df$name, legend=T, title_app="(test)")
options(repr.plot.width=10, repr.plot.height=5)
multiplot(p1_1, p1_2, cols=2)


# Differential expression

Differential expression is performed by applying an ANOVA comparing the samples from condition 'a' and condition 'b'. This is performed both without and with compensation for the batch factor.

In [ ]:
sig_vectors <- list()
q_vectors <- list()

In [ ]:
calculate_anova <- function(row, cond, batch=NULL) {
    
    if (!is.null(batch)) {
        anova_df <- data.frame(cbind(Intensity=unlist(row), Cond=cond, Batch=batch))        
        av <- aov(Intensity~Cond+Batch, anova_df)        
    }
    else {
        anova_df <- data.frame(cbind(Intensity=unlist(row), Cond=cond))
        av <- aov(Intensity~Cond, anova_df)                
    }
    
    av_summary <- summary(av)
    p_val <- av_summary[[1]]["Cond", "Pr(>F)"]
    p_val
}

In [ ]:
get_truth_vector <- function(measured_pos_vect, actually_pos_vect) {
    
    truth_vector <- c()
    
    for (i in 1:length(measured_pos_vect)) {
        
        measured_pos <- measured_pos_vect[i]
        actually_pos <- actually_pos_vect[i]
        
        if (measured_pos && actually_pos) {
            outcome <- "TP"
        }
        else if (!measured_pos && !actually_pos) {
            outcome <- "TN"
        }
        else if (measured_pos && !actually_pos) {
            outcome <- "FP"
        }
        else if (!measured_pos && actually_pos) {
            outcome <- "FN"
        }
        else {
            stop(paste("Unknown state, measured_pos:", measured_pos, 
                       ", actually_pos:", actually_pos))
        }
        
        truth_vector <- c(truth_vector, outcome)
    }
    
    truth_vector
}

print_truth_summary <- function(truth_vector) {
    
    truth_table <- table(truth_vector)
    print(truth_table)
    
    FN <- truth_table["FN"]
    FP <- truth_table["FP"]
    TN <- truth_table["TN"]
    TP <- truth_table["TP"]
    
    specificity <- TN / (TN + FP)
    precision <- TP / (TP + FP)
    recall <- TP / (TP + FN)
    fscore <- 2 * (precision * recall) / (precision + recall)
    
    print(paste("Precision:", round(precision, 3)))
    print(paste("Specificity:", round(specificity, 3)))
    print(paste("Recall:", round(recall, 3)))
    print(paste("F-score:", round(fscore, 3)))
}

## Without batch correction

Investigating differential expression on data without any batch-effect compensation.

In [ ]:
print("Without batch correction")

p_vals <- unlist(apply(data_df, 1, calculate_anova, cond=design_df$condition))
q_vals <- as.numeric(p.adjust(p_vals, method="BH"))
sig_indices <- which(q_vals < 0.1)

print(paste("P-vals below 0.1:", length(p_vals[which(p_vals < 0.1)])))
print(paste("Spike-in count:", length(spike_col[spike_col])))
print(paste("Passing FDR threshold:", length(q_vals[which(q_vals < 0.1)])))

truth_vector <- get_truth_vector(q_vals < 0.1, spike_col)
print_truth_summary(truth_vector)

sig_vectors[["default"]] <- truth_vector
q_vectors[["default"]] <- q_vals

## With batch correction

Investigating differential expression on data while including the batch effect as a factor used for the ANOVA model.

In [ ]:
print("With batch correction")

p_vals <- unlist(apply(data_df, 1, calculate_anova, cond=design_df$condition, batch=design_df$batch))
q_vals <- as.numeric(p.adjust(p_vals, method="BH"))
sig_indices <- which(q_vals < 0.1)

print(paste("P-vals below 0.1:", length(p_vals[which(p_vals < 0.1)])))
print(paste("Spike-in count:", length(spike_col[spike_col])))
print(paste("Passing FDR threshold:", length(q_vals[which(q_vals < 0.1)])))

truth_vector <- get_truth_vector(q_vals < 0.1, spike_col)
print_truth_summary(truth_vector)
sig_vectors[["batch_corr"]] <- truth_vector
q_vectors[["batch_corr"]] <- q_vals

# Run ComBat to adjust

ComBat is an algorithm attempting to identify and remove batch effects from expression data. It produces a new dataset which then in turn is used for differential expression between the conditions.

## ComBat modelling condition

Here, the condition is included in the ComBat modelling.

In [ ]:
modcombat <- model.matrix(~1+condition, data=design_df)
modcombat

In [ ]:
combat_data_df <- ComBat(dat=data_df, batch=design_df$batch, mod=modcombat, par.prior=TRUE, prior.plots=TRUE)

In [ ]:
head(combat_data_df)

In [ ]:
p1_1 <- plot_pca(data_df, design_df, 1, 2, "batch", colors=4, custom_names=design_df$name, legend=F, title_app="Normal")
p1_2 <- plot_pca(combat_data_df, design_df, 1, 2, "batch", colors=4, custom_names=design_df$name, legend=T, title_app="ComBat")
options(repr.plot.width=10, repr.plot.height=5)
multiplot(p1_1, p1_2, cols=2)


### Statistics

In [ ]:
print("ComBat statistics")

p_vals <- unlist(apply(combat_data_df, 1, calculate_anova, cond=design_df$condition))
q_vals <- as.numeric(p.adjust(p_vals, method="BH"))
sig_indices <- which(q_vals < 0.1)

print(paste("P-vals below 0.1:", length(p_vals[which(p_vals < 0.1)])))
print(paste("Spike-in count:", length(spike_col[spike_col])))
print(paste("Passing FDR threshold:", length(q_vals[which(q_vals < 0.1)])))

truth_vector <- get_truth_vector(q_vals < 0.1, spike_col)
print_truth_summary(truth_vector)
sig_vectors[["combat"]] <- truth_vector
q_vectors[["combat"]] <- q_vals

## ComBat not modelling biological factor

In [ ]:
modcombat <- model.matrix(~1, data=design_df)
head(modcombat)

In [ ]:
only_int_combat_data_df <- ComBat(dat=data_df, batch=design_df$batch, mod=modcombat, par.prior=TRUE, prior.plots=TRUE)

In [ ]:
p1_1 <- plot_pca(data_df, design_df, 1, 2, "batch", colors=4, custom_names=design_df$name, legend=F, title_app = "Normal")
p1_2 <- plot_pca(only_int_combat_data_df, design_df, 1, 2, "batch", colors=4, custom_names=design_df$name, legend=T, title_app="ComBat")
options(repr.plot.width=10, repr.plot.height=5)
multiplot(p1_1, p1_2, cols=2)


### Statistics

In [ ]:
print("ComBat statistics (only intercept)")

p_vals <- unlist(apply(only_int_combat_data_df, 1, calculate_anova, cond=design_df$condition))
q_vals <- as.numeric(p.adjust(p_vals, method="BH"))
sig_indices <- which(q_vals < 0.1)

print(paste("P-vals below 0.1:", length(p_vals[which(p_vals < 0.1)])))
print(paste("Spike-in count:", length(spike_col[spike_col])))
print(paste("Passing FDR threshold:", length(q_vals[which(q_vals < 0.1)])))

truth_vector <- get_truth_vector(q_vals < 0.1, spike_col)
print_truth_summary(truth_vector)
sig_vectors[["combat_no_factor"]] <- truth_vector
q_vectors[["combat_no_factor"]] <- q_vals

# Estimate surrogate variation

In [ ]:
mod <- model.matrix(~condition, data=design_df)
mod0 <- model.matrix(~1, data=design_df)
n.sv <- num.sv(data_df, mod, method="leek")
n.sv

In [ ]:
data_m <- as.matrix(data_df)
typeof(as.matrix(data_df))

In [ ]:
svobj <- sva(data_m, mod, mod0, n.sv=n.sv)

In [ ]:
str(svobj)

In [ ]:
pValues <- f.pvalue(data_m, mod, mod0)
qValues <- p.adjust(pValues, method="BH")
print(paste("Total:", length(pValues)))
print(paste("Sig: ", length(qValues[which(qValues < 0.1)])))

In [ ]:
modSv <- cbind(mod, svobj$sv)
mod0Sv <- cbind(mod0, svobj$sv)
pValuesSv <- f.pvalue(data_m, modSv, mod0Sv)
qValuesSv <- p.adjust(pValuesSv, method="BH")
print(paste("Sig:", length(qValuesSv[which(qValuesSv < 0.1)])))

In [ ]:
truth_vector <- get_truth_vector(measured_pos_vect=qValuesSv < 0.1, actually_pos_vect=spike_col)
print_truth_summary(truth_vector)
sig_vectors[["sva"]] <- truth_vector
q_vectors[["sva"]] <- qValuesSv

# Summarize significance

In [ ]:
print(paste("True positives:", length(spike_col[spike_col])))

print("Sig counts:")
for (name in names(q_vectors)) {
    q_vector <- q_vectors[[name]]
    print(paste0(name, ": ", length(q_vector[which(q_vector < 0.1)])))
}

In [ ]:
for (sig_v in names(sig_vectors)) {
    print(paste("===", sig_v, "==="))
    print_truth_summary(sig_vectors[[sig_v]])
}